In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

import openai 
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from llama_index.llms import OpenAI
from llama_index import VectorStoreIndex, SimpleDirectoryReader



### Load any text data

In [3]:
docs = SimpleDirectoryReader("samples/").load_data()

### Create embeddings

In [4]:
index = VectorStoreIndex.from_documents(docs,show_progress=True)

d:\Anaconda\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 17/17 [00:03<00:00,  4.45it/s]


### Query the document

In [13]:
query_engine = index.as_query_engine()
resp = query_engine.query("O czym jest ten dokument, opisz go w kilku zdaniach.")
print(resp)

Ten dokument jest analizą obowiązków związanych z dokumentacją cen transferowych dla spółki. Analiza została przeprowadzona przez firmę KPMG, która grupowała transakcje przedstawione przez Zleceniodawcę jako transakcje jednorodne. Dokument zawiera prośbę o weryfikację i potwierdzenie prawidłowości grupowania transakcji oraz informuje o możliwości kontaktu w razie jakichkolwiek wątpliwości.


### Save embeddings locally

In [11]:
index.storage_context.persist()

### Create index from stored embeddings

In [16]:
from llama_index import StorageContext,load_index_from_storage

storeage_context = StorageContext.from_defaults(persist_dir="storage/")
index = load_index_from_storage(storeage_context)

### Change default models

In [20]:
from llama_index import ServiceContext, set_global_service_context

In [21]:
llm = OpenAI(model="gpt-3.5-turbo",temperature=0,max_tokens=100)

serv_context = ServiceContext.from_defaults(llm=llm)

index = load_index_from_storage(storeage_context,service_context=serv_context)

In [22]:
index.as_query_engine().query("O czym jest ten dokument, opisz go w kilku zdaniach.")

Response(response='Ten dokument jest analizą obowiązków związanych z dokumentacją cen transferowych. Zawiera informacje dotyczące korzystania z ulgi B+R oraz grupowania transakcji jako transakcji jednorodnych. Dokument został przygotowany przez firmę KPMG i prosi o weryfikację i potwierdzenie prawidłowości grupowania transakcji.', source_nodes=[NodeWithScore(node=TextNode(id_='f6d4fdcb-a70b-4985-a7fa-cb62155c432d', embedding=None, metadata={'page_label': '5', 'file_name': 'TC_Interstal_Analiza obowiązków TP_memorandum_20221230.pdf', 'file_path': 'samples\\TC_Interstal_Analiza obowiązków TP_memorandum_20221230.pdf', 'creation_date': '2023-10-31', 'last_modified_date': '2023-10-31', 'last_accessed_date': '2023-10-31'}, excluded_embed_metadata_keys=['creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='7449eaf5-1922-404b-

# HF custom LLM

In [24]:
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts import PromptTemplate

system_prompt = """<|SYSTEM|>#
- Act as a helpfull assistant
- Patiently answer every question"""

query_wrapper = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [33]:
import torch

llm = HuggingFaceLLM(system_prompt=system_prompt,
                     model="meta-llama/Llama-2-7b-chat-hf",
                     tokenizer="meta-llama/Llama-2-7b-chat-hf",
                     query_wrapper_prompt=query_wrapper,
                    #  model_kwargs={"torch_dtype":torch.float16}
                     )

AttributeError: 'str' object has no attribute 'config'